In [2]:
import sys
sys.path.append("..")
import cr_interface as cri
import numpy as np
import pandas as pd
import sklearn as skl

/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
results = cri.load_results()
metadata = cri.load_metadata()

In [4]:
results.sort(key=lambda d: d['test_accuracy'])
best_result = results[-1]
best_result['test_accuracy']

'0.8821138'

In [5]:
selected_result = best_result

In [6]:
def get_result_df(result, training=False):
    if training:
        cr_codes = map(cri.extract_cr_code, result['training_images'])
    else:
        cr_codes = map(cri.extract_cr_code, result['predictions'].keys())
        
    cr_codes = list(set(cr_codes))
    
    for i, cr_code in enumerate(cr_codes):
        cr_codes[i] = 'D00' + cr_code[3:]

    extract_dataset = lambda cr_code: cri.parse_cr_code(cr_code)[0]
    extract_pid = lambda cr_code: cri.parse_cr_code(cr_code)[1]
    extract_phase = lambda cr_code: cri.parse_cr_code(cr_code)[2]
    extract_slice = lambda cr_code: cri.parse_cr_code(cr_code)[3]

    datasets = list(map(extract_dataset, cr_codes))
    pids = list(map(extract_pid, cr_codes))
    phases = list(map(extract_phase, cr_codes))
    slices = list(map(extract_slice, cr_codes))

    extract_label = lambda cr_code: metadata[cr_code]['label']
    labels = list(map(extract_label, cr_codes))
    
    # make tri labels
    for i, label in enumerate(labels):
        if label in ['ap', 'bs', 'md']:
            labels[i] = 'in'
        
    d = dict(
            cr_code=cr_codes,
            dataset=datasets,
            pid=pids,
            phase=phases,
            slice=slices,
            label=labels,
    )
    
    if not training:
        extract_prediction = lambda cr_code: result['predictions']['testing_{}'.format(cr_code)]['prediction']
        predictions = list(map(extract_prediction, cr_codes))
        d['prediction'] = predictions
        

    return pd.DataFrame(d)

In [9]:
df = get_result_df(selected_result, training=False)
df = df.drop_duplicates()
pd.crosstab(df['label'], df['prediction'])
df.describe()

,dataset,pid,phase,slice
count,246.0,246.000000,246.0,246.000000
mean,0.0,41629.861789,0.0,5.821138
std,0.0,2625.996473,0.0,3.855624
min,0.0,35501.000000,0.0,0.000000
25%,0.0,39501.000000,0.0,3.000000
50%,0.0,42451.000000,0.0,6.000000
75%,0.0,43701.000000,0.0,9.000000
max,0.0,45301.000000,0.0,16.000000


In [10]:
df = get_result_df(selected_result, training=False)
df = df.drop_duplicates()
pd.crosstab(df['label'], df['prediction'])

precision = {} # 정밀도
recall = {} # 재현율

lp = df[['label', 'prediction']]
for label in set(df['label']):
    true_positives = lp[lambda e: (e.label == label) & (e.prediction == label)]
    truths = lp[lambda e: e.label == label]
    positives = lp[lambda e: e.prediction == label]
    
    precision[label] = len(true_positives) / len(positives)
    recall[label] = len(true_positives) / len(truths)

summary_df = pd.DataFrame(dict(
    precision=precision,
    recall=recall
))
summary_df

,precision,recall
in,0.901163,0.933735
oap,0.906977,0.795918
obs,0.741935,0.741935
